Installing neseccary packages

In [45]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importing necessary items and libraries

In [46]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [47]:
import json
import re

Loading the dataset

In [48]:
with open('news.article.json', 'r', encoding='utf-8') as file:
    articles = json.load(file)

Initiating the t-5 model and tokenizer

In [49]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

C:\Users\91758\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [50]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Text processing

In [95]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.lower()

keywords = [
    "Hamas", "Jersalem", "Palestine", "israel", "Gaza", "attack", "war"
]
# Filter articles that have 'content' and match any of the keywords
filtered_articles = []
for article in articles:
        if any(keyword in article["articleBody"] for keyword in keywords) :
            filtered_articles.append(article)
        
# Filter articles that have 'content' and match any of the keywords

# Clean the articles
cleaned_articles = [
    {'title': clean_text(article.get('title', '')), 'content': clean_text(article['articleBody'])} 
    for article in filtered_articles
]

In [63]:
len(cleaned_articles)

27098

Defining various functions to be used later

In [64]:
def prepare_input(question, context):
    
    return f"answer the following question based on the context: {question} context: {context}"

In [75]:
def generate_answer(question, context, model, tokenizer, max_length=512, temperature=0.1):
    input_text = prepare_input(question, context)
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=max_length)

    outputs = model.generate(
        inputs, 
        max_length=max_length, 
        num_return_sequences=1, 
        temperature=temperature
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [96]:
# Combine the content of all cleaned articles for context
combined_context = "".join(article.get('articleBody', "") for article in cleaned_articles)

# Define the question
question = " hamas vs israel?"

# Generate the answer
answer = generate_answer(question, combined_context, model, tokenizer)
print(answer)

Israel
